# Example Use

In [ ]:
import nd_python as nd_p # Rust package run through python
import pandas as pd
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import time

plt.rcParams.update({'font.size': 24})
# these can be unassigned and will be automatically used by package
n = 10_000 # 10,000 node network (should use larger networks in analysis)
buckets = np.array([5,12,18,30,40,50,60,70]) # age groups : 0-4, 5-11, 12-17, 18-29, 30-39, 40-49, 50-59, 60-69, 70+
partitions = [0.058*n, 0.145*n, 0.212*n, 0.364*n, 0.497*n, 0.623*n, 0.759*n, 0.866*n, n] # prop of population in each group (UK census)

# data set used (POLYMOD, CoMix1)
datas = ['poly','comix1']
# models used to create networks (SBM, HBM with dPlN distribution)
models = ['sbm','dpln']

# define ODE model
def sir_model(t, y, beta, gamma, N):
    S, I, R = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    return [dSdt, dIdt, dRdt]


for data in datas:
    # import data as dataframe
    print(f'{data}\n')
    df = pd.read_csv(f'input_data/{data}.csv',sep=',')

    plt.figure(figsize=(12,10))

    for model in models:
        print(f'{model}:')
        start = time.process_time()
        
        # fit to data set
        egos, contact_matrix, params = nd_p.fit_to_data(df, dist_type=model, buckets=buckets)
        t1 = time.process_time() - start
        print(f'Data fit ({np.round(t1,4)} seconds)')
        
        # build network 
        network = nd_p.build_network(n, partitions=partitions, contact_matrix=contact_matrix, params=params, dist_type=model)
        t2 = time.process_time() - start - t1
        print(f'Network Build ({np.round(t2,4)} seconds)')
        
        # simulate outbreak
        for i in range(100):
            outbreak = nd_p.simulate(network=network, partitions=partitions, beta=0.021 if data==datas[0] else 0.03, inv_gamma=7, prop_infec=10/n, scaling='none')
            plt.plot([a for i,a in enumerate(outbreak['t']) if a!=0 or i==0], [a[1] for a in outbreak['SIR']], 'tab:blue' if model==models[0] else 'tab:green', alpha = 0.3)
        print(f'Outbreak Simulation and plot ({time.process_time() - start - t2} seconds)\n\n')
        # just to make legend prettier
        plt.plot([-1],[-1],'tab:blue' if model==models[0] else 'tab:green', linewidth=6, label=model)

    # ODE comparator
    solution = sc.integrate.solve_ivp(
        sir_model,
        [0,200],
        [n-10, 10, 0],
        args=(2/7, 1/7, n),
        dense_output=True
    )
    t = np.linspace(0,200,400)
    S, I, R = solution.sol(t)

    #plotting details
    plt.plot(t, I, 'k')
    plt.xlim([0,180])
    plt.ylim([0, plt.gca().get_ylim()[1]])
    plt.xlabel('days')
    plt.ylabel('Infections')
    plt.title(f'{data}')
    plt.legend()
    plt.show()